# 리더보드 1.51736점 코드 공개 (ResNet 기반 모델)

## 사용할 패키지

In [1]:
import numpy as np
import pandas as pd
import os
# import matplotlib.pyplot as plt
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Conv2D, Add, BatchNormalization, concatenate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

## 데이터 로드

In [2]:
submission = pd.read_csv('data/sample_submission.csv')
train_files = os.listdir('data/train')
# Python method "listdir()" returns a list containing the names of the entries in the directory given by path.

In [3]:
submission

,id,px_1,px_2,px_3,px_4,px_5,px_6,px_7,px_8,px_9,...,px_1591,px_1592,px_1593,px_1594,px_1595,px_1596,px_1597,px_1598,px_1599,px_1600
0,029858_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,029858_02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,029858_03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,029858_05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,029858_07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2411,031287_08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2412,031288_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2413,031288_02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2414,031288_08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
train_files

['subset_010462_01.npy',
 'subset_010462_02.npy',
 'subset_010462_03.npy',
 'subset_010462_04.npy',
 'subset_010462_05.npy',
 'subset_010462_06.npy',
 'subset_010462_07.npy',
 'subset_010463_01.npy',
 'subset_010463_02.npy',
 'subset_010463_03.npy',
 'subset_010463_04.npy',
 'subset_010463_05.npy',
 'subset_010463_06.npy',
 'subset_010463_07.npy',
 'subset_010463_08.npy',
 'subset_010463_09.npy',
 'subset_010463_10.npy',
 'subset_010463_11.npy',
 'subset_010463_12.npy',
 'subset_010464_01.npy',
 'subset_010464_02.npy',
 'subset_010464_03.npy',
 'subset_010464_04.npy',
 'subset_010464_05.npy',
 'subset_010464_06.npy',
 'subset_010464_07.npy',
 'subset_010464_08.npy',
 'subset_010464_09.npy',
 'subset_010464_10.npy',
 'subset_010464_11.npy',
 'subset_010464_12.npy',
 'subset_010465_01.npy',
 'subset_010465_02.npy',
 'subset_010465_03.npy',
 'subset_010465_04.npy',
 'subset_010465_05.npy',
 'subset_010468_01.npy',
 'subset_010468_02.npy',
 'subset_010468_03.npy',
 'subset_010468_04.npy',


In [31]:
train_files_batch1 = []

for i in range(1000):
    train_files_batch1.append(train_files[30000+i])

In [32]:
train_files_batch1

['subset_017119_07.npy',
 'subset_017119_08.npy',
 'subset_017119_09.npy',
 'subset_017119_10.npy',
 'subset_017119_11.npy',
 'subset_017119_12.npy',
 'subset_017120_01.npy',
 'subset_017120_02.npy',
 'subset_017120_03.npy',
 'subset_017120_04.npy',
 'subset_017123_01.npy',
 'subset_017123_02.npy',
 'subset_017123_03.npy',
 'subset_017123_04.npy',
 'subset_017123_05.npy',
 'subset_017123_06.npy',
 'subset_017123_07.npy',
 'subset_017124_01.npy',
 'subset_017124_02.npy',
 'subset_017124_03.npy',
 'subset_017124_04.npy',
 'subset_017124_05.npy',
 'subset_017124_06.npy',
 'subset_017124_07.npy',
 'subset_017124_08.npy',
 'subset_017124_09.npy',
 'subset_017124_10.npy',
 'subset_017124_11.npy',
 'subset_017124_12.npy',
 'subset_017125_01.npy',
 'subset_017125_02.npy',
 'subset_017125_03.npy',
 'subset_017125_04.npy',
 'subset_017125_05.npy',
 'subset_017125_06.npy',
 'subset_017125_07.npy',
 'subset_017125_08.npy',
 'subset_017125_09.npy',
 'subset_017125_10.npy',
 'subset_017125_11.npy',


In [33]:
train = []
for file in train_files_batch1:
    try:
        data = np.load('data/train/'+file).astype('float32')
        train.append(data)
    except:
        continue
        
test = []
for sub_id in submission['id']:
    data = np.load('data/test/'+'subset_'+sub_id+'.npy').astype('float32')
    test.append(data)

In [41]:
for i in range(1000):
    if train[i][39][13] == 0:
        train[i][39][13]

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [9]:
train = np.array(train)
test = np.array(test)
# numpy.array() : return an array objecty satisfying the specified requirements.

In [10]:
np.shape()

array([[[[1.74677109e+02, 9.75098343e+01, 2.03541229e+02, ...,
          1.59641464e+02, 5.57419205e+00, 0.00000000e+00],
         [1.74911652e+02, 9.66895828e+01, 2.03997940e+02, ...,
          1.59641464e+02, 5.57419205e+00, 0.00000000e+00],
         [1.73597321e+02, 9.79812927e+01, 2.04016159e+02, ...,
          1.59641464e+02, 5.57419205e+00, 0.00000000e+00],
         ...,
         [1.73038483e+02, 9.59997482e+01, 1.99570999e+02, ...,
          1.61207397e+02, 4.79637623e+00, 0.00000000e+00],
         [1.73871918e+02, 9.62958984e+01, 1.99392685e+02, ...,
          1.61265976e+02, 4.81756973e+00, 0.00000000e+00],
         [1.73283081e+02, 9.52175674e+01, 1.99537445e+02, ...,
          1.61307953e+02, 4.79751778e+00, 0.00000000e+00]],

        [[1.73488480e+02, 9.73820572e+01, 2.01760101e+02, ...,
          1.59690399e+02, 5.69830656e+00, 0.00000000e+00],
         [1.74805115e+02, 9.69153137e+01, 2.01477768e+02, ...,
          1.59690399e+02, 5.69830656e+00, 0.00000000e+00],
        